<a href="https://colab.research.google.com/github/W-Mrt/ScratchNeuralNetworks/blob/main/scratchneuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import pickle
from math import erf
from matplotlib import pyplot as plt
from abc import ABC, abstractmethod

In [3]:
#import data from Kaggle
import json, os

# 1. Install the Kaggle API client
!pip install kaggle --quiet

In [4]:
# 2. Set your Kaggle credentials
# Load the file
with open('kaggle.json') as f:
    creds = json.load(f)

# Set environment variables
os.environ['KAGGLE_USERNAME'] = creds['username']
os.environ['KAGGLE_KEY'] = creds['key']

# 3. Import and authenticate
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()  # reads env variables correctly

# 4. Download and unzip the MNIST CSV dataset
api.dataset_download_files(
    'oddrationale/mnist-in-csv',
    path='mnist-data',
    unzip=True,
    quiet=False
)

Dataset URL: https://www.kaggle.com/datasets/oddrationale/mnist-in-csv


100%|██████████| 15.2M/15.2M [00:00<00:00, 910MB/s]

In [5]:
# 1. Load CSVs into pandas and prepare NumPy arrays

train = pd.read_csv('mnist-data/mnist_train.csv')
test = pd.read_csv('mnist-data/mnist_test.csv')

# 2. Concatenate data
full = pd.concat([train, test], ignore_index=True)

# 3. Shuffle dataset
full = full.sample(frac=1, random_state=42).reset_index(drop=True)

# 4. Split into new train/test (e.g., 80/20)
split = int(0.8 * len(full))
train_new = full.iloc[:split]
test_new  = full.iloc[split:]

# 5. Extract features and labels
X_train = train_new.drop('label', axis=1).values.astype(np.float32) / 255.0
Y_train = train_new['label'].values.astype(np.int64)
X_test  = test_new.drop('label', axis=1).values.astype(np.float32) / 255.0
Y_test  = test_new['label'].values.astype(np.int64)

# Optional - Transpose for NumPy network
X_train = X_train.T         # (784, 60000)
Y_train = Y_train.reshape(1, -1)
X_test = X_test.T           # (784, 10000)
Y_test = Y_test.reshape(1, -1)

In [6]:
# Define NN model and utilities

# --- Initialization ---
def init_params(input_dim=784, hidden_dim=64, output_dim=10):
    np.random.seed(1)
    W1 = np.random.randn(hidden_dim, input_dim) * 0.01
    b1 = np.zeros((hidden_dim, 1))
    W2 = np.random.randn(output_dim, hidden_dim) * 0.01
    b2 = np.zeros((output_dim, 1))
    return W1, b1, W2, b2

# --- Activations ---
def relu(Z):
    return np.maximum(0, Z)

def drelu(Z):
    return (Z > 0).astype(float)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / expZ.sum(axis=0, keepdims=True)

# --- One-hot encoding ---
def one_hot(Y, num_classes=10):
    m = Y.size
    Y_oh = np.zeros((num_classes, m))
    Y_oh[Y, np.arange(m)] = 1
    return Y_oh

# --- Forward pass ---
def forward(X, W1, b1, W2, b2):
    Z1 = W1.dot(X) + b1
    A1 = relu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# --- Loss ---
def compute_loss(A2, Y_oh):
    m = Y_oh.shape[1]
    loss = -np.sum(Y_oh * np.log(A2 + 1e-9)) / m
    return loss

# --- Backward pass ---
def backward(X, Y_oh, Z1, A1, A2, W2):
    m = X.shape[1]
    dZ2 = A2 - Y_oh
    dW2 = (1/m) * dZ2.dot(A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = W2.T.dot(dZ2)
    dZ1 = dA1 * drelu(Z1)
    dW1 = (1/m) * dZ1.dot(X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

# --- Parameter update ---
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    return W1, b1, W2, b2

# --- Prediction ---
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward(X, W1, b1, W2, b2)
    return np.argmax(A2, axis=0)

In [7]:
def train(X, Y, epochs=30, lr=0.1, hidden_dim=64):
    n_x, m = X.shape
    Y_oh = one_hot(Y.flatten(), num_classes=10)

    #---- GRADIENT DESCENT
    W1, b1, W2, b2 = init_params(input_dim=n_x, hidden_dim=hidden_dim)
    for epoch in range(1, epochs+1):
        Z1, A1, Z2, A2 = forward(X, W1, b1, W2, b2)
        loss = compute_loss(A2, Y_oh)

        dW1, db1, dW2, db2 = backward(X, Y_oh, Z1, A1, A2, W2)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, lr)

    #---- show stats epoch, loss, accuracy
        if epoch % 5 == 0 or epoch == 1:
            preds = predict(X, W1, b1, W2, b2)
            acc = np.mean(preds == Y.flatten())
            print(f"Epoch {epoch:2d} │ Loss: {loss:.4f} │ Accuracy: {acc*100:.2f}%")

    return W1, b1, W2, b2

# Train the model
W1, b1, W2, b2 = train(X_train, Y_train, epochs=100, lr=0.1, hidden_dim=64)

# Evaluate on test set
preds_test = predict(X_test, W1, b1, W2, b2)
test_acc = np.mean(preds_test == Y_test.flatten())
print(f"\nTest set accuracy: {test_acc*100:.2f}%")

Epoch  1 │ Loss: 2.3024 │ Accuracy: 10.44%
Epoch  5 │ Loss: 2.2999 │ Accuracy: 25.47%
Epoch 10 │ Loss: 2.2963 │ Accuracy: 42.68%
Epoch 15 │ Loss: 2.2914 │ Accuracy: 53.23%
Epoch 20 │ Loss: 2.2845 │ Accuracy: 58.30%
Epoch 25 │ Loss: 2.2744 │ Accuracy: 59.77%
Epoch 30 │ Loss: 2.2596 │ Accuracy: 59.64%
Epoch 35 │ Loss: 2.2379 │ Accuracy: 58.50%
Epoch 40 │ Loss: 2.2065 │ Accuracy: 57.39%
Epoch 45 │ Loss: 2.1624 │ Accuracy: 57.41%
Epoch 50 │ Loss: 2.1028 │ Accuracy: 59.05%
Epoch 55 │ Loss: 2.0259 │ Accuracy: 62.08%
Epoch 60 │ Loss: 1.9316 │ Accuracy: 65.35%
Epoch 65 │ Loss: 1.8216 │ Accuracy: 68.08%
Epoch 70 │ Loss: 1.7001 │ Accuracy: 69.63%
Epoch 75 │ Loss: 1.5732 │ Accuracy: 70.31%
Epoch 80 │ Loss: 1.4482 │ Accuracy: 70.85%
Epoch 85 │ Loss: 1.3314 │ Accuracy: 71.81%
Epoch 90 │ Loss: 1.2269 │ Accuracy: 73.14%
Epoch 95 │ Loss: 1.1359 │ Accuracy: 74.68%
Epoch 100 │ Loss: 1.0577 │ Accuracy: 76.09%

Test set accuracy: 76.33%


In [8]:
class Initializers:
    """Weight initialization strategies for dense layers."""

    @staticmethod
    def xavier_uniform(shape):
        fan_in, fan_out = shape[1], shape[0]
        limit = np.sqrt(6.0 / (fan_in + fan_out))
        return np.random.uniform(-limit, limit, size=shape)

    @staticmethod
    def xavier_normal(shape):
        fan_in, fan_out = shape[1], shape[0]
        std = np.sqrt(2.0 / (fan_in + fan_out))
        return np.random.randn(*shape) * std

    @staticmethod
    def he_normal(shape):
        fan_in = shape[1]
        std = np.sqrt(2.0 / fan_in)
        return np.random.randn(*shape) * std

    @staticmethod
    def lecun_normal(shape):
        """Designed for SELU activation: variance = 1/fan_in."""
        fan_in = shape[1]
        std = np.sqrt(1.0 / fan_in)
        return np.random.randn(*shape) * std

    @staticmethod
    def orthogonal(shape):
        """Orthogonal initialization via SVD/X decomposition."""
        a = np.random.randn(*shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        return u.reshape(shape) if u.shape == shape else v.reshape(shape)

    @staticmethod
    def fixup_linear(shape, layer_idx, total_layers, m=2):
        """
        Fixup initialization for a linear (dense) layer in ResNet-like blocks.
        - Zero init for classification layer and last layer in each residual branch.
        - He init scaled by L^(-1/(2m−2)) for other layers in residual branches.
        - m = number of weight layers in a block/residual branch (usually 2).
        """
        fan_in = shape[1]
        if layer_idx == total_layers - 1:
            # Zero the final linear layer or last layer in residual branch
            return np.zeros(shape)
        # He init base
        W = np.random.randn(*shape) * np.sqrt(2.0 / fan_in)
        # Scale according to Fixup theory: L^(−1/(2m−2))
        scale = total_layers ** (-1.0 / (2 * m - 2))
        return W * scale

In [9]:
class ActivationFunctions:
    """Common activation functions and their derivatives."""

    @staticmethod
    def sigmoid(Z):
        A = 1 / (1 + np.exp(-Z))
        return A

    @staticmethod
    def dsigmoid(Z):
        A = ActivationFunctions.sigmoid(Z)
        return A * (1 - A)

    @staticmethod
    def tanh(Z):
        return np.tanh(Z)

    @staticmethod
    def dtanh(Z):
        return 1 - np.tanh(Z) ** 2

    @staticmethod
    def relu(Z):
        return np.maximum(0, Z)

    @staticmethod
    def drelu(Z):
        return (Z > 0).astype(float)

    @staticmethod
    def leaky_relu(Z, alpha=0.01):
        return np.where(Z > 0, Z, alpha * Z)

    @staticmethod
    def dleaky_relu(Z, alpha=0.01):
        return np.where(Z > 0, 1, alpha)

    @staticmethod
    def elu(Z, alpha=1.0):
        return np.where(Z > 0, Z, alpha * (np.exp(Z) - 1))

    @staticmethod
    def delu(Z, alpha=1.0):
        return np.where(Z > 0, 1, alpha * np.exp(Z))

    @staticmethod
    def selu(Z):
        alpha = 1.67326
        lam = 1.0507
        return lam * np.where(Z > 0, Z, alpha * (np.exp(Z) - 1))

    @staticmethod
    def dselu(Z):
        alpha = 1.67326
        lam = 1.0507
        return lam * np.where(Z > 0, 1, alpha * np.exp(Z))

    @staticmethod
    def softplus(Z):
        return np.log1p(np.exp(Z))

    @staticmethod
    def dsoftplus(Z):
        return 1 / (1 + np.exp(-Z))

    @staticmethod
    def swish(Z, beta=1.0):
        sig = 1 / (1 + np.exp(-beta * Z))
        return Z * sig

    @staticmethod
    def dswish(Z, beta=1.0):
        sig = 1 / (1 + np.exp(-beta * Z))
        return sig + Z * beta * sig * (1 - sig)

    @staticmethod
    def gelu(Z):
        # exact form: x * 0.5 * (1 + erf(x/sqrt(2)))
        return Z * 0.5 * (1 + erf(Z / np.sqrt(2)))

    @staticmethod
    def dgelu(Z):
        # derivative approximation can be implemented if needed
        sig = 1 / (1 + np.exp(-Z))
        return sig * (1 + Z * (1 - sig))  # simplified approx.

    @staticmethod
    def softmax(Z):
        expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return expZ / expZ.sum(axis=0, keepdims=True)

In [10]:
class LRScheduler:
    """Learning rate scheduler with basic decay strategies."""

    @staticmethod
    def fixed(lr0, step):
        return lr0

    @staticmethod
    def time_decay(lr0, step, decay=1e-3):
        return lr0 / (1 + decay * step)

    @staticmethod
    def step_decay(lr0, step, drop=0.5, epochs_drop=10):
        return lr0 * drop**(step // epochs_drop)

    @staticmethod
    def exp_decay(lr0, step, decay=1e-2):
        return lr0 * np.exp(-decay * step)

class LRRL:
    """
    Choose learning rate dynamically using a multi-armed bandit (e.g., Exp3).
    Arms = discrete set of candidate learning rates.
    Rewards = measured improvements during training "episodes".
    """

    def __init__(self, lr_list, gamma=0.1):
        self.lr_list = lr_list
        self.K = len(lr_list)
        self.weights = np.ones(self.K)
        self.gamma = gamma

    def get_lr(self):
        prob = (1 - self.gamma) * (self.weights / self.weights.sum()) + (self.gamma / self.K)
        idx = np.random.choice(self.K, p=prob)
        return self.lr_list[idx], idx

    def update(self, idx, reward):
        x = np.zeros(self.K); x[idx] = reward / self.weights[idx]
        self.weights *= np.exp((self.gamma / self.K) * x)

In [11]:
class SGDMomentum:
    """
    Stochastic Gradient Descent with Momentum.
    params: list of parameter arrays (e.g., [W1, b1, W2, b2])
    grads: list of corresponding gradients
    """

    def __init__(self, lr=0.01, beta=0.9, scheduler=None):
        self.lr = lr
        self.beta = beta
        self.scheduler = scheduler  # e.g., a learning rate schedule function
        self.v = None
        self.step = 0

    def update(self, params, grads):
        if self.v is None:
            # Initialize velocity terms with zeros
            self.v = [np.zeros_like(p) for p in params]

        # Retrieve current learning rate, possibly decayed
        lr = self.scheduler(self.lr, self.step) if self.scheduler else self.lr

        new_params = []
        for i, (p, g) in enumerate(zip(params, grads)):
            self.v[i] = self.beta * self.v[i] + (1 - self.beta) * g
            new_p = p - lr * self.v[i]
            new_params.append(new_p)

        self.step += 1
        return new_params

In [12]:
class RMSProp:
    """
    RMSProp optimizer.
    Maintains moving average of squared gradients and updates parameters adaptively.
    """

    def __init__(self, lr=0.001, decay=0.9, epsilon=1e-8, scheduler=None):
        self.lr = lr
        self.decay = decay  # often referred to as 'rho'
        self.epsilon = epsilon
        self.scheduler = scheduler
        self.s = None
        self.step = 0

    def update(self, params, grads):
        if self.s is None:
            self.s = [np.zeros_like(p) for p in params]

        lr_t = self.scheduler(self.lr, self.step) if self.scheduler else self.lr

        updated_params = []
        for p, g, si in zip(params, grads, self.s):
            si[:] = self.decay * si + (1 - self.decay) * (g ** 2)
            updated_params.append(p - lr_t * g / (np.sqrt(si) + self.epsilon))

        self.step += 1
        return updated_params

In [ ]:
class Adam:
    """
    Adam optimizer combining momentum and RMSProp.
    """

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, scheduler=None):
        """
        :param lr: learning rate (alpha)
        :param beta1: momentum decay rate (first moment)
        :param beta2: RMSProp decay rate (second moment)
        :param epsilon: small constant to avoid division by zero
        :param scheduler: optional learning-rate schedule function: fn(lr0, step) → lr_t
        """
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.scheduler = scheduler

        self.m = None  # First moment vector
        self.v = None  # Second moment vector
        self.step = 0

    def update(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]

        self.step += 1
        lr_t = self.scheduler(self.lr, self.step) if self.scheduler else self.lr

        updated_params = []
        for i, (p, g) in enumerate(zip(params, grads)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (g ** 2)

            m_hat = self.m[i] / (1 - self.beta1 ** self.step)
            v_hat = self.v[i] / (1 - self.beta2 ** self.step)

            updated = p - lr_t * m_hat / (np.sqrt(v_hat) + self.epsilon)
            updated_params.append(updated)

        return updated_params

In [ ]:
class ConjugateGradient:
    """
    Nonlinear Conjugate Gradient optimizer for neural networks.
    Requires:
      - loss_fn(flat_params) → scalar loss
      - grad_fn(flat_params) → flattened gradient
    Uses Polak-Ribiere update:
      d_t = -g_t + β_t d_{t-1}
    Includes line search stub for step size α determination.
    """
    def __init__(self):
        self.prev_grad = None
        self.prev_dir = None

    def update(self, flat_params, loss_fn, grad_fn):
        grad = grad_fn(flat_params)

        if self.prev_grad is None:
            direction = -grad
        else:
            y = grad - self.prev_grad
            beta = np.dot(grad, y) / (np.dot(self.prev_grad, self.prev_grad) + 1e-12)  # Polak-Ribiere
            direction = -grad + beta * self.prev_dir

        # Line search to find optimal alpha (stub)
        alpha = self.line_search(flat_params, direction, loss_fn)

        new_flat_params = flat_params + alpha * direction

        self.prev_grad = grad
        self.prev_dir = direction

        return new_flat_params

    def line_search(self, flat_params, direction, loss_fn):
        """
        Simple backtracking line search:
        α = max α ∈ {cⁿ α0} such that loss decreases adequately.
        """
        alpha = 1.0
        c = 0.5
        rho = 0.5
        loss0 = loss_fn(flat_params)
        grad_phi0 = np.dot(self.prev_grad, direction)
        while True:
            new_loss = loss_fn(flat_params + alpha * direction)
            if new_loss <= loss0 + c * alpha * grad_phi0:
                break
            alpha *= rho
            if alpha < 1e-6:
                break
        return alpha

In [ ]:
class PINNOptimizer:
    """
    Physics-Informed Neural Network optimizer with gradient enhancement (gPINN).
    Only uses finite differences – no autograd dependencies.
    """

    def __init__(self, base_opt, pinn_residual_fn, pinn_data, weight_pde=1.0, weight_grad=0.1, fd_eps=1e-4):
        """
        :param base_opt: optimizer (SGD, Adam, etc.)
        :param pinn_residual_fn: fn(params, X_f) -> residuals vector of shape (N_f,)
        :param pinn_data: collocation points (X_f)
        :param weight_pde: weight on residual loss
        :param weight_grad: weight on gradient-of-residual loss
        :param fd_eps: finite difference epsilon
        """
        self.base = base_opt
        self.res_fn = pinn_residual_fn
        self.X_f = pinn_data
        self.w_pde = weight_pde
        self.w_grad = weight_grad
        self.fd_eps = fd_eps

    def pinn_res(self, params):
        """
        Computes residual r(x) = u''(x) - f(x) at collocation points via finite differences.
        """
        W1, b1, W2, b2 = params  # for a simple 1-hidden-layer network
        def u(x):
            z1 = W1.dot(x) + b1
            a1 = np.tanh(z1)
            return (W2.dot(a1) + b2).flatten()

        x = self.X_f
        u_plus = u(x + self.fd_eps)
        u_minus = u(x - self.fd_eps)
        u0 = u(x)
        u_xx = (u_plus - 2*u0 + u_minus) / (self.fd_eps**2)
        return u_xx - self.f(x)

    def pinn_grads_fd(self, params):
        """
        Finite-difference approximation of PDE residual gradients.
        Approximates ∂(Σ r(x)^2)/∂θ for each param using finite difference.
        """
        grads = []
        r0 = self.res_fn(params, self.X_f)  # baseline residuals
        N_f = r0.shape[0]
        for W in params:
            gW = np.zeros_like(W)
            it = np.nditer(W, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                idx = it.multi_index
                orig = W[idx]
                W[idx] = orig + self.fd_eps
                r1 = self.res_fn(params, self.X_f)
                W[idx] = orig - self.fd_eps
                r2 = self.res_fn(params, self.X_f)
                W[idx] = orig
                # gradient of sum(residual^2) ≈ 2 * residual * dr/dw
                dr = (r1 - r2) / (2 * self.fd_eps)
                gW[idx] = np.sum(2 * r0 * dr)
                it.iternext()
            grads.append(gW)
        return grads

    def update(self, params, data_grads):
        """
        Combine data-driven gradients with PDE residual & gradient-of-residual gradients,
        then perform an optimizer step.
        """
        pinn_grads = self.pinn_grads_fd(params)

        combined = [
            dg + self.w_pde * pg + self.w_grad * pg
            for dg, pg in zip(data_grads, pinn_grads)
        ]
        return self.base.update(params, combined)


In [ ]:
class HessianFreeOptimizer:
    def __init__(self, model, loss_fn, max_iter=10, cg_iter=10, tol=1e-6):
        """
        Initialize the Hessian-Free optimizer.

        Parameters:
        - model: The neural network model.
        - loss_fn: The loss function.
        - max_iter: Maximum number of outer iterations.
        - cg_iter: Number of conjugate gradient iterations.
        - tol: Tolerance for convergence.
        """
        self.model = model
        self.loss_fn = loss_fn
        self.max_iter = max_iter
        self.cg_iter = cg_iter
        self.tol = tol

    def compute_gradients(self, X, y):
        """
        Compute the gradients of the loss function with respect to the model parameters.

        Parameters:
        - X: Input data.
        - y: True labels.

        Returns:
        - gradients: Gradients of the loss function.
        """
        # Forward pass
        y_pred = self.model.forward(X)
        # Compute loss
        loss = self.loss_fn(y_pred, y)
        # Backward pass
        gradients = self.model.backward(X, y)
        return gradients, loss

    def compute_hvp(self, X, y, vector):
        """
        Compute the Hessian-vector product using the conjugate gradient method.

        Parameters:
        - X: Input data.
        - y: True labels.
        - vector: Vector to compute the Hessian-vector product with.

        Returns:
        - hvp: Hessian-vector product.
        """
        def hvp_fn(v):
            # Compute the gradient of the loss with respect to the model parameters
            gradients, _ = self.compute_gradients(X, y)
            # Compute the Hessian-vector product
            hvp = np.dot(gradients, v)
            return hvp

        # Use conjugate gradient method to compute the Hessian-vector product
        hvp = self.conjugate_gradient(hvp_fn, vector)
        return hvp

    def conjugate_gradient(self, A, b):
        """
        Solve Ax = b using the conjugate gradient method.

        Parameters:
        - A: Function that computes the matrix-vector product.
        - b: Right-hand side vector.

        Returns:
        - x: Solution vector.
        """
        x = np.zeros_like(b)
        r = b - A(x)
        p = r
        rs_old = np.dot(r, r)
        for _ in range(self.cg_iter):
            Ap = A(p)
            alpha = rs_old / np.dot(p, Ap)
            x += alpha * p
            r -= alpha * Ap
            rs_new = np.dot(r, r)
            if np.sqrt(rs_new) < self.tol:
                break
            p = r + (rs_new / rs_old) * p
            rs_old = rs_new
        return x

    def step(self, X, y):
        """
        Perform a single optimization step.

        Parameters:
        - X: Input data.
        - y: True labels.
        """
        # Compute the gradient and loss
        gradients, loss = self.compute_gradients(X, y)
        # Compute the Hessian-vector product
        hvp = self.compute_hvp(X, y, gradients)
        # Update the model parameters
        self.model.update_parameters(hvp)
        return loss

In [ ]:
class BaseOptimizer:
    def step(self, params, grads, *args, **kwargs):
        """
        Unified interface for optimizer steps.
        Subclasses or wrapped optimizers should implement this.
        """
        raise NotImplementedError("Optimizer must implement step().")

class UpdateAdapterOptimizer(BaseOptimizer):
    def __init__(self, impl):
        self.impl = impl

    def step(self, params, grads, *args, **kwargs):
        # Adapts your existing update(params, grads) signature
        return self.impl.update(params, grads)

class OptimizerFactory:
    @staticmethod
    def create(name, **kwargs):
        name = name.lower()
        if name == "sgd":
            return UpdateAdapterOptimizer(SGDMomentum(**kwargs))
        elif name == "rmsprop":
            return UpdateAdapterOptimizer(RMSProp(**kwargs))
        elif name == "adam":
            return UpdateAdapterOptimizer(Adam(**kwargs))
        elif name in ("cg", "conjugategradient"):
            return ConjugateGradient()
        elif name == "pinn":
            base = kwargs.pop("base_opt")
            pinn_res = kwargs.pop("pinn_residual_fn")
            pinn_data = kwargs.pop("pinn_data")
            return UpdateAdapterOptimizer(
                PINNOptimizer(base, pinn_res, pinn_data, **kwargs)
            )
        elif name in ("hessianfree", "hf"):
            model = kwargs.pop("model")
            loss_fn = kwargs.pop("loss_fn")
            return HessianFreeOptimizer(model, loss_fn, **kwargs)
        else:
            raise ValueError(f"Unknown optimizer: {name}")

In [ ]:
class ModelPersistence:
    @staticmethod
    def save(model, path):
        with open(path,"wb") as f:
            pickle.dump(model, f)

    @staticmethod
    def load(path):
        with open(path,"rb") as f:
            return pickle.load(f)

class NeuralNet:
    def __init__(self, layers, activations, loss_fn, optimizer: BaseOptimizer):
        self.layers = layers
        self.activations = activations
        self.loss_fn = loss_fn
        self.optimizer = optimizer  # Any object implementing step()
        self.weights, self.biases = self._initialize_parameters()

    def _initialize_parameters(self):
        import numpy as np
        Ws, bs = [], []
        for i in range(len(self.layers)-1):
            Ws.append(np.random.randn(self.layers[i], self.layers[i+1]))
            bs.append(np.zeros((1, self.layers[i+1])))
        return Ws, bs

    def forward(self, X):
        self.zs, self.activations_cache = [], [X]
        for W, b, act_fn in zip(self.weights, self.biases, self.activations):
            z = self.activations_cache[-1] @ W + b
            self.zs.append(z)
            self.activations_cache.append(act_fn(z))
        return self.activations_cache[-1]

    def backward_logic(Z1, A1, A2, W2, X, Y_oh):
        """
        Compute the gradients of the loss function with respect to weights and biases.

        Parameters:
        - Z1: Linear activation of the first layer
        - A1: Activation of the first layer
        - A2: Activation of the second layer (output)
        - W2: Weights of the second layer
        - X: Input data
        - Y_oh: One-hot encoded true labels

        Returns:
        - grads_W: List of gradients for weights
        - grads_b: List of gradients for biases
        """
        m = X.shape[1]  # Number of examples

        # Compute gradients for the second layer
        dZ2 = A2 - Y_oh
        dW2 = (1 / m) * np.dot(dZ2, A1.T)
        db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

        # Compute gradients for the first layer
        dA1 = np.dot(W2.T, dZ2)
        dZ1 = dA1 * drelu(Z1)  # Assuming drelu is the derivative of ReLU
        dW1 = (1 / m) * np.dot(dZ1, X.T)
        db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

        # Return gradients as lists
        grads_W = [dW1, dW2]
        grads_b = [db1, db2]
        return grads_W, grads_b

    def backward(self, X, y):
        """
        Perform the backward pass to compute gradients with respect to weights and biases.

        Args:
            X (numpy.ndarray): Input data of shape (input_size, batch_size).
            y (numpy.ndarray): True labels of shape (output_size, batch_size).

        Returns:
            tuple: A tuple containing:
                - grads_W (list): List of gradients with respect to weights for each layer.
                - grads_b (list): List of gradients with respect to biases for each layer.
        """
        # Compute gradients using the backpropagation logic
        grads_W, grads_b = self.backward_logic(self.zs, self.activations_cache, y)

        return grads_W, grads_b

    def train_epoch(self, X, y):
        params = self.get_flat_params()  # however you combine W & b
        grads = self.get_flat_grads(X, y)
        new_params = self.optimizer.step(params, grads)
        self.set_flat_params(new_params)

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.train_epoch(X, y)
            if epoch % 10 == 0:
                loss_val = self.loss_fn(self.forward(X), y)
                print(f"Epoch {epoch}, Loss {loss_val:.4f}")

class IncrementalTrainer:
    def __init__(self, model: NeuralNet):
        self.model = model

    def update(self, new_X, new_y, epochs=10):
        print("Updating with new data...")
        self.model.train(new_X, new_y, epochs)

    def retrain(self, full_X, full_y, epochs=50):
        print("Retraining from scratch...")
        self.model.weights, self.model.biases = self.model._initialize_parameters()
        self.model.train(full_X, full_y, epochs)